In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx

import rpy2 as rp
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pyreadr

import os
import time
import h5py
# import mmdet
import scanpy
# import ssam
import pickle
import random
# import pyarrow
import dgl
import ast

import anndata as ad
import scanpy as sc
import scipy.sparse as sp
from sklearn import metrics
import multiprocessing as mp
from GraphST import GraphST
from h5py import File

from tqdm import tqdm

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, matthews_corrcoef, mean_squared_error

# from captum.attr import IntegratedGradients
# from captum.attr import LayerConductance
# from captum.attr import NeuronConductance

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

# import torch_geometric

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils

from skimage import io
from skimage import img_as_float
from scipy import ndimage

# from ssam.utils import corr, calc_ctmap, calc_corrmap, flood_fill, calc_kde
from scipy.spatial.distance import cdist

from collections import OrderedDict

from packaging import version
import subprocess

2024-12-16 11:34:27.420826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-16 11:34:27.420864: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-16 11:34:27.421710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-16 11:34:28.153185: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from matplotlib.image import imread
import json
from pathlib import Path, PurePath

def put_image(adata, path, library_id, arr_col=True):
    source_image_path = path
    adata.uns["spatial"] = dict()
    # from h5py import File
    
    # with File(path / count_file, mode="r") as f:
    #     attrs = dict(f.attrs)
    # if library_id is None:
    #     library_id = str(attrs.pop("library_ids")[0], "utf-8")
    adata.uns["spatial"][library_id] = dict()
    tissue_positions_file = path + "/spatial/tissue_positions_list.csv"
    files = dict(
        tissue_positions_file=tissue_positions_file,
        scalefactors_json_file=path + "/spatial/scalefactors_json.json",
        hires_image=path + "/spatial/tissue_hires_image.png",
        lowres_image=path + "/spatial/tissue_lowres_image.png",
    )
    # check if files exists, continue if images are missing
    # for f in files.values():
    #     if not f.exists():
    #         if any(x in str(f) for x in ["hires_image", "lowres_image"]):
    #             logg.warning(
    #                 f"You seem to be missing an image file.\n"
    #                 f"Could not find '{f}'."
    #             )
    #         else:
    #             raise OSError(f"Could not find '{f}'")
    
    adata.uns["spatial"][library_id]["images"] = dict()
    for res in ["hires", "lowres"]:
        try:
            adata.uns["spatial"][library_id]["images"][res] = imread(
                str(files[f"{res}_image"])
            )
        except Exception:
            raise OSError(f"Could not find '{res}_image'")
    
    # read json scalefactors
    adata.uns["spatial"][library_id]["scalefactors"] = json.loads(
        Path(files["scalefactors_json_file"]).read_text()
    )
    
    # adata.uns["spatial"][library_id]["metadata"] = {
    #     k: (str(attrs[k], "utf-8") if isinstance(attrs[k], bytes) else attrs[k])
    #     for k in ("chemistry_description", "software_version")
    #     if k in attrs
    # }
    
    # read coordinates
    positions = pd.read_csv(
        files["tissue_positions_file"],
        header= None,
        index_col=0,
    )
    positions.columns = [
        "in_tissue",
        "array_row",
        "array_col",
        "pxl_col_in_fullres",
        "pxl_row_in_fullres",
    ]
    if arr_col:
        adata.obs = adata.obs.join(positions, how="left")
        
        adata.obsm["spatial"] = adata.obs[
            ["pxl_row_in_fullres", "pxl_col_in_fullres"]
        ].to_numpy()
        adata.obs.drop(
            columns=["pxl_row_in_fullres", "pxl_col_in_fullres"],
            inplace=True,
        )
    
    # put image path in uns
    # if source_image_path is not None:
    #     # get an absolute path
    #     source_image_path = str(Path(source_image_path).resolve())
    #     adata.uns["spatial"][library_id]["metadata"]["source_image_path"] = str(
    #         source_image_path
    #     )

    return adata

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=1)

In [4]:
os.getcwd()

'/workspace/volume/spatial_integration/GNN_based/combined/New/Reworked_flow'

<h1>Take Matrix

In [5]:
dataset_name = 'coronal' #Designated output name ['DLPFC_combine', 'Mouse_combine']

In [6]:
sample_folder = 'dataset/samples/' #sample df_logcounts (step 2 output) folder
embed_folder = 'dataset/' #sample spatial embedding (step 3 output) folder

In [7]:
type = 'rds' #feather or rds

# sample_names = ['coronal_section1', 'coronal_section2'] #coronal dataset sample_id
sample_names = ['151507','151672', '151673'] #DLPFC sample_id

count_dict = {}
spat_dict = {}

for i, folder_name in enumerate(sample_names, start=1):
    print(f'reading {folder_name}...')
    # df_read = pyreadr.read_r(sample_folder + f"df_logcounts_{folder_name}.rds")

    if type == 'rds' :
        # read from rds     
        df_read = pyreadr.read_r(sample_folder + f"df_logcounts_{folder_name}.rds")
        count_dict[folder_name] = df_read[None]
    elif type == 'feather':
        # read feather
        count_dict[folder_name] = pd.read_feather(sample_folder + f'/df_logcounts_{folder_name}.feather')
    else:
        print('unspecified format type')

    spat_read = np.load(embed_folder + f"encoded_{folder_name}_unfil.npy")
    
    spat_dict[folder_name] = pd.DataFrame(spat_read.reshape((spat_read.shape[0], spat_read.shape[-1])))
    spat_dict[folder_name].columns = count_dict[folder_name].columns

    ind_list = count_dict[folder_name].index
    new_list = []
    for i in range(len(ind_list)):
        new_list.append(ind_list[i]+'-spat')
    spat_dict[folder_name].index = new_list
  
print(count_dict.keys())
print(spat_dict.keys())

reading 151507...
reading 151672...
reading 151673...
dict_keys(['151507', '151672', '151673'])
dict_keys(['151507', '151672', '151673'])


In [8]:
combined_all =  pd.concat(count_dict.values(), axis=1)

In [9]:
combined_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
embed_dict = {}
for sample in sample_names:
    embed_dict[sample] = pd.concat([count_dict[sample], spat_dict[sample]],axis = 0)
embed_all = pd.concat(embed_dict.values(), axis=1)
embed_all.fillna(0,inplace=True)

In [12]:
row_names = embed_all.index
row_names_df = row_names.to_frame(index=False)
row_names_df.to_csv(f'dataset/index_name_{dataset_name}.csv')

In [14]:
embed_all.columns = pd.io.common.dedup_names(embed_all.columns, is_potential_multiindex=False)

In [14]:
print(f'saving to dataset/embed_all_{dataset_name}.feather')
embed_all.to_feather(f'dataset/embed_all_{dataset_name}.feather')
for sample in sample_names:
    embed_dict[sample].fillna(0,inplace=True)
    print(f'saving to dataset/{sample}_{dataset_name}.feather')
    embed_dict[sample].to_feather(f'dataset/{sample}_{dataset_name}.feather')

saving to dataset/embed_all_DLPFC.feather
saving to dataset/151507_DLPFC.feather
saving to dataset/151672_DLPFC.feather
saving to dataset/151673_DLPFC.feather


<h1>END</h1>

----